# Generate Data

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
from mlchecks import Dataset, validate_dataset, validate_dataset_or_dataframe
from mlchecks.checks.integrity.rare_format_detection import RareFormatDetection


In [2]:
pd.set_option('display.max_columns', None)

In [3]:
np.random.seed(42)

In [4]:
datelist = pd.date_range(start=datetime.today(), periods=200, freq='D', normalize=True)
s_date = pd.Series([d.strftime('%Y-%m-%d') for d in datelist], name='date')

emaillist = [''.join(np.random.choice(a=list('abcdefghijklmnopqrstuvwxyz'), p=[1/26]*26, size=np.random.choice(a=[6,7,8], p=[0.2, 0.5, 0.3]))) + '@gmail.com' for x in range(200)]
s_email = pd.Series(emaillist, name='email')

df = pd.DataFrame([s_date, s_email]).T

In [5]:
df.head()

,date,email
0,2021-10-17,ytpeebw@gmail.com
1,2021-10-18,sazvfee@gmail.com
2,2021-10-19,nlhpdhj@gmail.com
3,2021-10-20,ufnpbpe@gmail.com
4,2021-10-21,yzvhcr@gmail.com


# Run Check - Find no changes

In [6]:
ds = Dataset(df)
check = RareFormatDetection()

check.run(dataset=ds)

c:\users\nirhu\documents\deepchecks\mlchecks\mlchecks\base\dataset.py:57: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._features = [x for x in df.columns if x not in {label, index, date}]
c:\users\nirhu\documents\deepchecks\mlchecks\mlchecks\base\dataset.py:67: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._cat_features = self.infer_categorical_features()


{'date': Empty DataFrame
Columns: []
Index: [], 'email': Empty DataFrame
Columns: []
Index: []}

# Induce changes to data

In [7]:
# Change dates to different format:
df['date'].loc[0:2] = [datetime.strptime(d, '%Y-%m-%d').strftime('%Y-%b-%d') for d in df['date'].loc[0:2]]

# Change emails to have mistakes in format
df['email'].loc[[0,1]] = ['myname@gmail.com1', 'myname@gmail.co']

In [8]:
df.head()

,date,email
0,2021-Oct-17,myname@gmail.com1
1,2021-Oct-18,myname@gmail.co
2,2021-Oct-19,nlhpdhj@gmail.com
3,2021-10-20,ufnpbpe@gmail.com
4,2021-10-21,yzvhcr@gmail.com


# Run Check - Find rare formats

In [9]:
ds = Dataset(df)
check = RareFormatDetection()

check.run(dataset=ds)

c:\users\nirhu\documents\deepchecks\mlchecks\mlchecks\base\dataset.py:57: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._features = [x for x in df.columns if x not in {label, index, date}]
c:\users\nirhu\documents\deepchecks\mlchecks\mlchecks\base\dataset.py:67: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._cat_features = self.infer_categorical_features()


{'date':                                              digits only format (ignoring letters)  \
ratio of rare patterns to common patterns                                    1.52%   
common formats                                                        [0000-00-00]   
examples for values in common formats                                 [2021-10-21]   
values in rare formats                     [2021-Oct-17, 2021-Oct-18, 2021-Oct-19]   

                                          sequences of digits only format (ignoring letters)  \
ratio of rare patterns to common patterns                                              1.52%   
common formats                                                                  [____-__-__]   
examples for values in common formats                                           [2021-10-21]   
values in rare formats                               [2021-Oct-17, 2021-Oct-18, 2021-Oct-19]   

                                          sequences of letters only format (ignoring letters)  \
ratio of rare patterns to common patterns                                              1.52%    
common formats                                                                          [--]    
examples for values in common formats                                           [2021-10-21]    
values in rare formats                               [2021-Oct-17, 2021-Oct-18, 2021-Oct-19]    

                                                         digits and letters format  \
ratio of rare patterns to common patterns                                    1.52%   
common formats                                                        [0000-00-00]   
examples for values in common formats                                 [2021-10-21]   
values in rare formats                     [2021-Oct-17, 2021-Oct-18, 2021-Oct-19]   

                                          digits and letters format (case sensitive)  \
ratio of rare patterns to common patterns                                      1.52%   
common formats                                                          [0000-00-00]   
examples for values in common formats                                   [2021-10-21]   
values in rare formats                       [2021-Oct-17, 2021-Oct-18, 2021-Oct-19]   

                                                               any sequence format  
ratio of rare patterns to common patterns                                    1.52%  
common formats                                                        [0000-00-00]  
examples for values in common formats                                 [2021-10-21]  
values in rare formats                     [2021-Oct-17, 2021-Oct-18, 2021-Oct-19]  , 'email':                                           digits only format (ignoring letters)  \
ratio of rare patterns to common patterns                                 0.50%   
common formats                                                             [@.]   
examples for values in common formats                       [nlhpdhj@gmail.com]   
values in rare formats                                      [myname@gmail.com1]   

                                          sequences of digits only format (ignoring letters)  \
ratio of rare patterns to common patterns                                              0.50%   
common formats                                                                          [@.]   
examples for values in common formats                                    [nlhpdhj@gmail.com]   
values in rare formats                                                   [myname@gmail.com1]   

                                                                   digits and letters format  \
ratio of rare patterns to common patterns                                              1.01%   
common formats                             [XXXXXXX@XXXXX.XXX, XXXXXXXX@XXXXX.XXX, XXXXXX...   
examples for values in common formats      [ufnpbpe@gmail.com, qxmdstou@gmail.com, vssubj...   
value